# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [2]:
# # # EXAMPLE (from LLM) — Auth + Project/Region (commented; write your own cell using the prompt)
# from google.colab import auth
# auth.authenticate_user()

# import os
# PROJECT_ID = input("Enter your GCP Project ID: ").strip()
# REGION = "us-central1"  # keep consistent; change if instructed
# os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
# print("Project:", PROJECT_ID, "| Region:", REGION)

# # Set active project for gcloud/BigQuery CLI
# !gcloud config set project $GOOGLE_CLOUD_PROJECT
# !gcloud config get-value project
# # Done: Auth + Project/Region set

In [3]:
# Prompt output
from google.colab import auth
auth.authenticate_user() # Authenticates the Colab environment to Google Cloud

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip() # Prompt for and store the GCP Project ID
REGION = "us-central1"  # Set the default region; change if necessary

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID # Export the Project ID as an environment variable

# Set the active project for gcloud commands
!gcloud config set project $GOOGLE_CLOUD_PROJECT
print("Project:", PROJECT_ID, "| Region:", REGION)

# Done: Auth + Project/Region set

Enter your GCP Project ID: sunlit-plasma-471119-s7
Updated property [core/project].
Project: sunlit-plasma-471119-s7 | Region: us-central1


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [4]:
# Verification: Print active project and region
!gcloud config get-value project
print(f"Region: {REGION}")

sunlit-plasma-471119-s7
Region: us-central1


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

We set 'PROJECT_ID' and 'REGION' at the beginning to ensure consistency, cost management, and to avoid errors. If we don't do this you could encounter errors in your code and incur unexpected costs in different regions.

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [5]:
# # EXAMPLE (from LLM) — Kaggle setup (commented)
# # from google.colab import files
# # print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
# # uploaded = files.upload()
# #
# # import os
# # os.makedirs('/root/.kaggle', exist_ok=True)
# # with open('/root/.kaggle/kaggle.json', 'wb') as f:
# #     f.write(uploaded[list(uploaded.keys())[0]])
# # os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only
# #
# # !kaggle --version

In [6]:
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only permissions for security

!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


In [7]:
# Verification: Show Kaggle CLI help output
!kaggle --help | head -n 20

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

\**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

We require strict '0600' permissions because we want only the owner to have read and write access to the API token. We avoid risks such as unauthorized access, token compromise, and accidental modification/deletion of the token.

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [8]:
# # EXAMPLE (from LLM) — Download & unzip (commented)
# # !mkdir -p /content/data/raw
# # !kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data
# # !unzip -o /content/data/*.zip -d /content/data/raw
# # # List CSV inventory
# # !ls -lh /content/data/raw/*.csv

In [9]:
# Create directory for raw data
!mkdir -p /content/data/raw

# Download the dataset using Kaggle CLI
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data

# Unzip the downloaded file into the raw data directory (overwrite if necessary)
!unzip -o /content/data/*.zip -d /content/data/raw

# List all CSV files in the raw data directory with their sizes
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
  0% 0.00/4.02M [00:00<?, ?B/s]
100% 4.02M/4.02M [00:00<00:00, 798MB/s]
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 19:36 /content/data/raw/users.csv
-rw-r--r-- 1 root root 8.9M A

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


In [10]:
# Verification: Assert the number of CSV files and print their names
import glob
csv_files = glob.glob('/content/data/raw/*.csv')
assert len(csv_files) == 6, f"Expected 6 CSV files, but found {len(csv_files)}"
print("Found 6 CSV files:")
for f in csv_files:
    print(f)

Found 6 CSV files:
/content/data/raw/movies.csv
/content/data/raw/watch_history.csv
/content/data/raw/search_logs.csv
/content/data/raw/users.csv
/content/data/raw/recommendation_logs.csv
/content/data/raw/reviews.csv


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

It's important to keep clean file inventory because it allows for auditing and reproducibility, verification, troubleshooting, and easy data discovery.

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [11]:
# # EXAMPLE (from LLM) — GCS staging (commented)
# # import uuid, os
# # bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
# # os.environ["BUCKET_NAME"] = bucket_name
# # !gcloud storage buckets create gs://$BUCKET_NAME --location=$REGION
# # !gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
# # print("Bucket:", bucket_name)
# # # Verify contents
# # !gcloud storage ls gs://$BUCKET_NAME/netflix/

In [12]:
import uuid
import os

# Create a unique bucket name with a random suffix
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name

# Create the GCS bucket
# Note: If you encounter an error related to the region,
# you might need to check available locations with '!gcloud storage locations list'
# and potentially update the REGION variable at the top of the notebook.
!gcloud storage buckets create gs://$BUCKET_NAME --location=US

# Upload all CSV files to the bucket under the 'netflix/' prefix
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/

# Print the bucket name and explain the benefits of staging
print(f"Created bucket: {bucket_name}")
print("\nBenefits of staging data in GCS:")
print("- Provides a consistent and versionable source for data.")
print("- Decouples data storage from processing, allowing for flexible use with various GCP services like BigQuery.")
print("- Enables central management and access control of data.")

Creating gs://mgmt467-netflix-d0667bcb/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-d0667bcb/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-d0667bcb/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-d0667bcb/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-d0667bcb/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-d0667bcb/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-d0667bcb/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-d0667bcb/netflix/watch_history.csv

Average throughput: 46.6MiB/s
Created bucket: mgmt467-netflix-d0667bcb

Benefits of staging data in GCS:
- Provides a consistent and versionable source for data.
- Decouples data storage from processing, allowing for flexible use with v

### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [13]:
# Verification: List contents of the netflix/ prefix in the bucket
import os
bucket_name = os.environ["BUCKET_NAME"]
!gcloud storage ls gs://$BUCKET_NAME/netflix/ --recursive --readable-sizes

gs://mgmt467-netflix-d0667bcb/netflix/:
gs://mgmt467-netflix-d0667bcb/netflix/README.md
gs://mgmt467-netflix-d0667bcb/netflix/movies.csv
gs://mgmt467-netflix-d0667bcb/netflix/recommendation_logs.csv
gs://mgmt467-netflix-d0667bcb/netflix/reviews.csv
gs://mgmt467-netflix-d0667bcb/netflix/search_logs.csv
gs://mgmt467-netflix-d0667bcb/netflix/users.csv
gs://mgmt467-netflix-d0667bcb/netflix/watch_history.csv


**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

1. Staging in GCS allows for easy accesibility by various google cloud services, which is essential for large datasets and complex workflows

2. Data in GCS persists independently of your Colab session. Even if you Colab runtime restarts, your data in GCS remains available, allowing you to easily reproduce it if needed as well.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [14]:
# # EXAMPLE (from LLM) — BigQuery dataset (commented)
# # DATASET="netflix"
# # # Attempt to create; ignore if exists
# # !bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

In [15]:
# # EXAMPLE (from LLM) — Load tables (commented)
# # tables = {
# #   "users": "users.csv",
# #   "movies": "movies.csv",
# #   "watch_history": "watch_history.csv",
# #   "recommendation_logs": "recommendation_logs.csv",
# #   "search_logs": "search_logs.csv",
# #   "reviews": "reviews.csv",
# # }
# # import os
# # for tbl, fname in tables.items():
# #   src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
# #   print("Loading", tbl, "from", src)
# #   !bq load --skip_leading_rows=1 --autodetect --source_format=CSV $DATASET.$tbl $src
# #
# # # Row counts
# # for tbl in tables.keys():
# #   !bq query --nouse_legacy_sql "SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `${GOOGLE_CLOUD_PROJECT}.netflix.{tbl}`".format(tbl=tbl)

In [16]:
# Cell A: Create BigQuery dataset
DATASET = "netflix"
# Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

BigQuery error in mk operation: Dataset 'sunlit-plasma-471119-s7:netflix'
already exists.
Dataset may already exist.


In [17]:
# Cell B: Load tables from GCS
tables = {
  "users": "users.csv",
  "movies": "movies.csv",
  "watch_history": "watch_history.csv",
  "recommendation_logs": "recommendation_logs.csv",
  "search_logs": "search_logs.csv",
  "reviews": "reviews.csv",
}
import os
for tbl, fname in tables.items():
  src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
  print("Loading", tbl, "from", src)
  !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

project = os.environ['GOOGLE_CLOUD_PROJECT']
# Row counts
for tbl in tables.keys():
  q = f"SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `{project}.{DATASET}.{tbl}`"
  print("Counting rows for", tbl)
  # Pass the whole query as a single argument to avoid backtick issues
  !bq query --nouse_legacy_sql --format=pretty "{q}"

Loading users from gs://mgmt467-netflix-d0667bcb/netflix/users.csv
Waiting on bqjob_r3fb9eec9acba2742_0000019a23b4eaf0_1 ... (1s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-d0667bcb/netflix/movies.csv
Waiting on bqjob_r2f2226944929855b_0000019a23b5002a_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-d0667bcb/netflix/watch_history.csv
Waiting on bqjob_r49797ef9a2a14298_0000019a23b5147f_1 ... (2s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-d0667bcb/netflix/recommendation_logs.csv
Waiting on bqjob_r3bde56b86fb49b50_0000019a23b52e22_1 ... (2s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-d0667bcb/netflix/search_logs.csv
Waiting on bqjob_r5c967c144614cabf_0000019a23b546fe_1 ... (1s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-d0667bcb/netflix/reviews.csv
Waiting on bqjob_r5822a0aead85b10_0000019a23b55b98_1 ... (1s) Current status: DONE   
Counting rows f

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

In [30]:
%%bigquery

SELECT 'users' AS table_name, COUNT(*) AS row_count FROM `sunlit-plasma-471119-s7.netflix.users`
UNION ALL
SELECT 'movies' AS table_name, COUNT(*) AS row_count FROM `sunlit-plasma-471119-s7.netflix.movies`
UNION ALL
SELECT 'watch_history' AS table_name, COUNT(*) AS row_count FROM `sunlit-plasma-471119-s7.netflix.watch_history`
UNION ALL
SELECT 'recommendation_logs' AS table_name, COUNT(*) AS row_count FROM `sunlit-plasma-471119-s7.netflix.recommendation_logs`
UNION ALL
SELECT 'search_logs' AS table_name, COUNT(*) AS row_count FROM `sunlit-plasma-471119-s7.netflix.search_logs`
UNION ALL
SELECT 'reviews' AS table_name, COUNT(*) AS row_count FROM `sunlit-plasma-471119-s7.netflix.reviews`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,movies,11440
1,users,113300
2,search_logs,291500
3,reviews,169950
4,watch_history,1155000
5,recommendation_logs,572000


## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [33]:
%%bigquery
-- Users: Total rows and % missing per column (region, plan_tier, age_band)
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(state_province IS NULL) miss_region,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `sunlit-plasma-471119-s7.netflix.users`
)
SELECT n,
       ROUND(SAFE_DIVIDE(100*miss_region, n),2) AS pct_missing_region,
       ROUND(SAFE_DIVIDE(100*miss_plan, n),2)   AS pct_missing_plan_tier,
       ROUND(SAFE_DIVIDE(100*miss_age, n),2)    AS pct_missing_age_band
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_region,pct_missing_plan_tier,pct_missing_age_band
0,113300,0.0,0.0,11.93


In [34]:
%%bigquery
-- Users: % plan_tier missing by region (examining MAR)
SELECT state_province,
       COUNT(*) AS n,
       ROUND(SAFE_DIVIDE(100*COUNTIF(subscription_plan IS NULL), COUNT(*)),2) AS pct_missing_plan_tier
FROM `sunlit-plasma-471119-s7.netflix.users`
GROUP BY state_province
ORDER BY pct_missing_plan_tier DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,state_province,n,pct_missing_plan_tier
0,Arizona,3971,0.0
1,Indiana,4114,0.0
2,New Jersey,3839,0.0
3,Newfoundland and Labrador,3344,0.0
4,Pennsylvania,3905,0.0
5,Alberta,3410,0.0
6,California,3993,0.0
7,Florida,3883,0.0
8,Georgia,3949,0.0
9,Illinois,3905,0.0


### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


In [35]:
%%bigquery
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(state_province IS NULL) miss_region,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `sunlit-plasma-471119-s7.netflix.users`
)
SELECT n,
       ROUND(SAFE_DIVIDE(100*miss_region, n),2) AS pct_missing_region,
       ROUND(SAFE_DIVIDE(100*miss_plan, n),2)   AS pct_missing_plan_tier,
       ROUND(SAFE_DIVIDE(100*miss_age, n),2)    AS pct_missing_age_band
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_region,pct_missing_plan_tier,pct_missing_age_band
0,113300,0.0,0.0,11.93


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

None of them having missing things.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [43]:
%%bigquery
-- Report duplicate groups on (user_id, movie_id, event_ts, device_type) with counts (top 20)
SELECT user_id, movie_id, device_type, COUNT(*) AS dup_count
FROM `sunlit-plasma-471119-s7.netflix.watch_history`
GROUP BY user_id, movie_id, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,device_type,dup_count
0,user_03310,movie_0640,Smart TV,44
1,user_00391,movie_0893,Laptop,44
2,user_03408,movie_0146,Desktop,33
3,user_08681,movie_0332,Laptop,33
4,user_02822,movie_0009,Desktop,33
5,user_01870,movie_0844,Laptop,33
6,user_09815,movie_0827,Laptop,33
7,user_07529,movie_0686,Laptop,33
8,user_09973,movie_0342,Desktop,33
9,user_01383,movie_0015,Desktop,33


In [42]:
%%bigquery
CREATE OR REPLACE TABLE `sunlit-plasma-471119-s7.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `sunlit-plasma-471119-s7.netflix.watch_history` h
)
WHERE rk = 1;

Query is running:   0%|          |

""


### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [45]:
%%bigquery
-- Compare row counts before and after deduplication
SELECT 'watch_history_raw' AS table_name, COUNT(*) AS row_count
FROM `sunlit-plasma-471119-s7.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count
FROM `sunlit-plasma-471119-s7.netflix.watch_history_dedup`;

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,watch_history_raw,1155000
1,watch_history_dedup,99878


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

Duplicates can arise from both natural and system-generated causes. Natural duplicates might occur when a user legitimately performs the same action multiple times in quick succession (e.g., refreshing a page). System-generated duplicates are more common in data pipelines and can result from errors in data collection, transmission, processing, or storage (e.g., a logging system sending the same event twice, or joins in a data transformation creating unintended duplicates).

Duplicates can corrupt labels and KPIs by artificially inflating counts and distorting distributions. For example:

- Labels: If you're building a model to predict user engagement based on watch history, duplicate entries for the same viewing session would make it seem like a user watched a movie for a much longer time than they actually did, leading to incorrect engagement labels.
- KPIs: Metrics like "total minutes watched," "number of unique viewers," or "completion rate" would be skewed by duplicate entries, leading to inaccurate reporting and potentially flawed business decisions.

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [46]:
%%bigquery
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `sunlit-plasma-471119-s7.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `sunlit-plasma-471119-s7.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

Query is running:   0%|          |

Downloading:   0%|          |

,outliers,total,pct_outliers
0,3516,99878,3.52


In [49]:
%%bigquery
CREATE OR REPLACE TABLE `sunlit-plasma-471119-s7.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `sunlit-plasma-471119-s7.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS minutes_watched_capped
FROM `sunlit-plasma-471119-s7.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after
WITH before AS (
  SELECT 'before' AS which, APPROX_QUANTILES(watch_duration_minutes, 5) AS q
  FROM `sunlit-plasma-471119-s7.netflix.watch_history_dedup`
),
after AS (
  SELECT 'after' AS which, APPROX_QUANTILES(minutes_watched_capped, 5) AS q
  FROM `sunlit-plasma-471119-s7.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;

Query is running:   0%|          |

Downloading:   0%|          |

,which,q
0,after,"[4.5, 24.6, 41.6, 61.6, 92.1, 204.0]"
1,before,"[0.2, 25.0, 41.6, 61.3, 92.0, 799.3]"


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [52]:
%%bigquery
WITH before AS (
  SELECT
    'before' AS which,
    MIN(watch_duration_minutes) AS min_watched,
    APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_watched,
    MAX(watch_duration_minutes) AS max_watched
  FROM `sunlit-plasma-471119-s7.netflix.watch_history_dedup`
),
after AS (
  SELECT
    'after' AS which,
    MIN(minutes_watched_capped) AS min_watched,
    APPROX_QUANTILES(minutes_watched_capped, 2)[OFFSET(1)] AS median_watched,
    MAX(minutes_watched_capped) AS max_watched
  FROM `sunlit-plasma-471119-s7.netflix.watch_history_robust`
)
SELECT * FROM before
UNION ALL
SELECT * FROM after;

Query is running:   0%|          |

Downloading:   0%|          |

,which,min_watched,median_watched,max_watched
0,after,4.5,51.0,204.0
1,before,0.2,50.8,799.3


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

Capping might be harmful when outliers represent genuine, important data points that could significantly influence the outcome you are trying to model. For example, in fraud detection, extreme values could be indicators of fraudulent activity, and capping them would obscure these signals.

Tree-based models like Decision Trees, Random Forests, and Gradient Boosting Machines (e.g., XGBoost, LightGBM) are generally less sensitive to outliers compared to linear models (like Linear Regression or Logistic Regression) or distance-based models (like K-Nearest Neighbors or K-Means). This is because tree-based models make decisions based on splitting data at certain thresholds rather than relying on distances or the overall distribution of the data, which can be heavily skewed by outliers.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [54]:
%%bigquery
-- Summarize flag_binge for sessions > 8 hours in watch_history_robust
SELECT
  COUNTIF(watch_duration_minutes > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_binge
FROM `sunlit-plasma-471119-s7.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,sessions_over_8h,total,pct_flag_binge
0,638,99878,0.64


In [55]:
%%bigquery
-- Summarize flag_age_extreme in users
-- Flag users with extreme ages (below 10 or above 100) based on parsing age_band.
SELECT
  COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
          SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_flag_age_extreme
FROM `sunlit-plasma-471119-s7.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,extreme_age_rows,total,pct_flag_age_extreme
0,1969,113300,1.74


In [56]:
%%bigquery
-- Summarize flag_duration_anomaly in movies
SELECT
  COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60) AS duration_anomaly_titles,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_duration_anomaly
FROM `sunlit-plasma-471119-s7.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,duration_anomaly_titles,total,pct_flag_duration_anomaly
0,253,11440,2.21


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [57]:
%%bigquery
-- Single compact summary query for anomaly flags
SELECT
  'flag_binge' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `sunlit-plasma-471119-s7.netflix.watch_history_robust`
UNION ALL
SELECT
  'flag_age_extreme' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_of_rows
FROM `sunlit-plasma-471119-s7.netflix.users`
UNION ALL
SELECT
  'flag_duration_anomaly' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `sunlit-plasma-471119-s7.netflix.movies`;


Query is running:   0%|          |

Downloading:   0%|          |

,flag_name,pct_of_rows
0,flag_binge,0.64
1,flag_age_extreme,1.74
2,flag_duration_anomaly,2.21


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

Based on the output of the last query, the flag_duration_anomaly is the most common with 2.21% of the rows flagged, followed by flag_age_extreme at 1.74%, and then flag_binge at 0.64%.

Regarding which to keep as a feature, it depends on the specific machine learning task. However, flag_age_extreme could be a useful feature for models predicting user behavior or content preferences, as very young or very old users might have different viewing habits than the general population. flag_binge could also be a strong indicator for engagement or churn prediction. flag_duration_anomaly might be less directly useful as a feature itself, but identifying these anomalies could be important for data cleaning or further investigation.

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
